In [1]:
from IPython.display import display, Math, Latex

In [2]:
a_x = var("n_x")
b_x = var("b_x")
a_y = var("n_y")
b_y = var("b_y")

assume(a_x, 'real')
assume(b_x, 'real')
assume(a_y, 'real')
assume(b_y, 'real')

assume(a_x, 'integer')
assume(b_x, 'integer')
assume(a_y, 'integer')
assume(b_y, 'integer')

assume(a_x > 0)
assume(b_x > 0)
assume(a_y > 0)
assume(b_y > 0)

x,y = var("x,y")

n = QQ['n'].0
j = QQ['j'].0

p = 4
p = 3
#p = 2
#p = 1

F = function("F")(x, y)
print(F)

F(x, y)


In [3]:
# https://www.journals.vu.lt/LMJ/article/view/20600/19701
# ФОРМУЛА СУММИРОВАНИЯ ЭЙЛЕРА—МАКЛОРЕНА ДЛЯ ФУНКЦИИ МНОГИХ ПЕРЕМЕННЫХ А.БИКЯЛИС
# VIII LIETUVOS MATEMATIKOS RINKINYS 4
# ЛИТОВСКИЙ МАТЕМАТИЧЕСКИЙ СБОРНИК
# 1968

# Далее,обе стороны этого равенства суммируем по всем целым m2 из интервала(z2,y2). Получаем

if True:
    B = lambda x, p : bernoulli_polynomial(x, p)
    P = lambda x, p : bernoulli_polynomial(x - floor(x), p)
    fact = lambda n : factorial(n)
else:
    B = function("B")(x,p)
    P = function("P")(x,p)
    fact = function("fact")(n)

def integral_R_p(f,symb,jx,hold, f_diff_symb_p=None):
    vx = var("v_"+str(symb))
    if None == f_diff_symb_p:
        deriv = f.diff(symb,p)
    else:
        deriv = f_diff_symb_p
    if logging:
        print("deriv", deriv)

    deriv_subs = deriv.subs(symb == vx+jx)
    if logging:
        print("deriv_subs", deriv_subs)

    int_f_diff_B_fact = integral(deriv_subs * bernoulli_polynomial(vx,p)/factorial(p), (vx,0,1), hold=hold)
    if logging:
        print("int_f_diff_B_fact", int_f_diff_B_fact)

    return int_f_diff_B_fact

def integral_R_p_shifted_first(f,symb,jx,hold):
    vx = var("v_"+str(symb))
    fn_shifted = f.subs(symb == vx + jx)
    if logging:
        print("fn_shifted", fn_shifted)

    deriv = fn_shifted.diff(vx,p)
    if logging:
        print("deriv", deriv)

    int_f_diff_B_fact = integral(deriv * B(x=vx,p=p)/fact(n=p), (vx, 0, 1), hold=hold)

    if logging:
        print("int_f_diff_B_fact", int_f_diff_B_fact)

    return int_f_diff_B_fact
    
    
def euler_maclaurin_R_p(f,symb,a,b,p, f_diff_symb_p=None, hold_int=True, hold_sum=True):
    if logging:
        print("f", f)
        print("symb,a,b", symb, a, b)
        if f_diff_symb_p is not None:
            print("f_diff_symb_p", f_diff_symb_p)
        print("")
    if use_int_R_p_j:
        jx = var('j_'+str(symb))
        assume(jx>0)
        from sage.calculus.calculus import symbolic_sum
        
        if change_R_p_order:
            vx = var("v_"+str(symb))
            # Change order of summation and integration
            if use_int_R_p_shifted_first:
                R_p = (-1)^(p+1)*integral(symbolic_sum(f.subs(symb == vx+jx).diff(vx,p)*B(x=vx,p=p)/fact(n=p), \
                                              jx, a, b-1, hold=hold_sum), \
                                          (vx,0,1), hold=hold_int)
            else:
                if None == f_diff_symb_p:
                    f_diff_symb_p = f.diff(symb,p)
                R_p = (-1)^(p+1)*integral(symbolic_sum(f_diff_symb_p.subs(symb == vx+jx)*B(x=vx,p=p)/fact(n=p), \
                                              jx, a, b-1, hold=hold_sum), \
                                          (vx,0,1), hold=hold_int)
        else:       
            if use_int_R_p_shifted_first:
                term = (-1)^(p+1)*integral_R_p_shifted_first(f=f, symb=symb, jx=jx, hold=hold_int)
                R_p = symbolic_sum(term, jx, a, b-1, hold=hold_sum)
            else:
                term = (-1)^(p+1)*integral_R_p(f=f, symb=symb, jx=jx, hold=hold_int, f_diff_symb_p=f_diff_symb_p)
                R_p = symbolic_sum(term, jx, a, b-1, hold=hold_sum)
            return R_p, term
    else:
        if not use_num_int:
            if None == f_diff_symb_p:
                f_diff_symb_p = f.diff(symb,p)
            if logging:
                print("(-1)^(p+1)*P(x=symb,p=p)/fact(n=p)", (-1)^(p+1)*P(x=symb,p=p)/fact(n=p))
            R_p = (-1)^(p+1)*integral(f_diff_symb_p * P(x=symb,p=p)/fact(n=p),(symb,a,b),hold=hold_int)
        else:
            R_p = (-1)^(p+1)*num_int(lambda symb: f_diff_symb_p * P(x=symb,p=p)/fact(n=p), a, b)
            
    return R_p

In [4]:
def sum_dfdx_bernoulis(f,symb,a,b,p):
    if logging:
        print("f", f)
        print("symb,a,b", symb, a, b)
    dfdx_a_bernoullis = []
    for k in range(1,1+p):
        dfdx_a_bernoullis += [(f.diff(symb,k-1))*(bernoulli(k)/factorial(k))]
        
    sum_dfdx_a_bernoullis = sum(dfdx_a_bernoullis)

    if logging:
        print("sum_dfdx_a_bernoullis", sum_dfdx_a_bernoullis)
        print("sum_dfdx_a_bernoullis(a)", sum_dfdx_a_bernoullis.subs(symb == a))
        if Infinity != b:
            print("sum_dfdx_a_bernoullis(b)", sum_dfdx_a_bernoullis.subs(symb == b))

    s = - sum_dfdx_a_bernoullis.subs(symb == a)
    if Infinity != b:
        s += sum_dfdx_a_bernoullis.subs(symb == b)
    return s


In [5]:
use_int_R_p_j=True
change_R_p_order=True
use_int_R_p_shifted_first=True
use_num_int = False
logging = True

In [6]:

use_int_R_p_j=True
# Sagemath 9.3 has bug diff of symbolic_sum
# https://trac.sagemath.org/ticket/32161

R_px = euler_maclaurin_R_p(F, x, a_x, b_x,p, f_diff_symb_p=F.diff(x,p))
print("R_px=", R_px)
display(Math(latex(R_px)))


sumy_R_px = sum_dfdx_bernoulis(R_px, y, a_y, b_y, p)
print("sumy_R_px=",sumy_R_px)
display(Math(latex(sumy_R_px)))

sum_R_p = sum_dfdx_bernoulis(euler_maclaurin_R_p(F, x, a_x, b_x,p, f_diff_symb_p=F.diff(x,p)), y, a_y, b_y, p)
print("sum_R_p=",sum_R_p)
display(Math(latex(sum_R_p)))


f F(x, y)
symb,a,b x n_x b_x
f_diff_symb_p diff(F(x, y), x, x, x)

R_px= integrate(sum(1/12*(2*v_x^3 - 3*v_x^2 + v_x)*D[0, 0, 0](F)(j_x + v_x, y), j_x, n_x, b_x - 1), v_x, 0, 1)


<IPython.core.display.Math object>

f integrate(sum(1/12*(2*v_x^3 - 3*v_x^2 + v_x)*D[0, 0, 0](F)(j_x + v_x, y), j_x, n_x, b_x - 1), v_x, 0, 1)
symb,a,b y n_y b_y
sum_dfdx_a_bernoullis -1/2*integrate(sum(1/12*(2*v_x^3 - 3*v_x^2 + v_x)*D[0, 0, 0](F)(j_x + v_x, y), j_x, n_x, b_x - 1), v_x, 0, 1) + 1/12*integrate(sum(1/12*(2*v_x^3 - 3*v_x^2 + v_x)*D[0, 0, 0, 1](F)(j_x + v_x, y), j_x, n_x, b_x - 1), v_x, 0, 1)
sum_dfdx_a_bernoullis(a) -1/2*integrate(sum(1/12*(2*v_x^3 - 3*v_x^2 + v_x)*D[0, 0, 0](F)(j_x + v_x, n_y), j_x, n_x, b_x - 1), v_x, 0, 1) + 1/12*integrate(sum(1/12*(2*v_x^3 - 3*v_x^2 + v_x)*D[0, 0, 0, 1](F)(j_x + v_x, n_y), j_x, n_x, b_x - 1), v_x, 0, 1)
sumy_R_px= 1/2*integrate(sum(1/12*(2*v_x^3 - 3*v_x^2 + v_x)*D[0, 0, 0](F)(j_x + v_x, n_y), j_x, n_x, b_x - 1), v_x, 0, 1) - 1/12*integrate(sum(1/12*(2*v_x^3 - 3*v_x^2 + v_x)*D[0, 0, 0, 1](F)(j_x + v_x, n_y), j_x, n_x, b_x - 1), v_x, 0, 1)


<IPython.core.display.Math object>

f F(x, y)
symb,a,b x n_x b_x
f_diff_symb_p diff(F(x, y), x, x, x)

f integrate(sum(1/12*(2*v_x^3 - 3*v_x^2 + v_x)*D[0, 0, 0](F)(j_x + v_x, y), j_x, n_x, b_x - 1), v_x, 0, 1)
symb,a,b y n_y b_y
sum_dfdx_a_bernoullis -1/2*integrate(sum(1/12*(2*v_x^3 - 3*v_x^2 + v_x)*D[0, 0, 0](F)(j_x + v_x, y), j_x, n_x, b_x - 1), v_x, 0, 1) + 1/12*integrate(sum(1/12*(2*v_x^3 - 3*v_x^2 + v_x)*D[0, 0, 0, 1](F)(j_x + v_x, y), j_x, n_x, b_x - 1), v_x, 0, 1)
sum_dfdx_a_bernoullis(a) -1/2*integrate(sum(1/12*(2*v_x^3 - 3*v_x^2 + v_x)*D[0, 0, 0](F)(j_x + v_x, n_y), j_x, n_x, b_x - 1), v_x, 0, 1) + 1/12*integrate(sum(1/12*(2*v_x^3 - 3*v_x^2 + v_x)*D[0, 0, 0, 1](F)(j_x + v_x, n_y), j_x, n_x, b_x - 1), v_x, 0, 1)
sum_R_p= 1/2*integrate(sum(1/12*(2*v_x^3 - 3*v_x^2 + v_x)*D[0, 0, 0](F)(j_x + v_x, n_y), j_x, n_x, b_x - 1), v_x, 0, 1) - 1/12*integrate(sum(1/12*(2*v_x^3 - 3*v_x^2 + v_x)*D[0, 0, 0, 1](F)(j_x + v_x, n_y), j_x, n_x, b_x - 1), v_x, 0, 1)


<IPython.core.display.Math object>